In [4]:
##sample script vector##

import arcpy

# set up working environment, need to specify just like in arcgis
#the r reads it as a raw string, which gets around the problem of there being a space in my folder name
arcpy.env.workspace = r"C:\Users\lfabr\OneDrive - UW-Madison\School\Madison\courses\2017-03-Fall\GEOG 378\labs\Lab7\data"

#this overwrites when you put out a file with the same name as one that already exits
arcpy.env.overwriteOutput = True

# set feature class name
fc1 = ".\data\sample_line.shp"
fc2 = ".\data\sample_polygon.shp"

# get feature class description
desc = arcpy.Describe(fc2)
print 'featureType:   ', desc.featureType
print 'shapeFieldName:', desc.shapeFieldName
print 'shapeType:     ', desc.shapeType
print 'extent:        ', desc.extent
print 'path:          ', desc.path
print


# get fields from feature class
#this is how you get the attribute information from the feature class
print '\nThe fields are:'
for field in desc.fields:
    fldName       = field.name
    fldWidth      = field.length  #is "Width" in OGR
    fldPrecision  = field.precision
    fldTypeS      = field.type    #is an integer in OGR

    values = (fldName,fldTypeS,fldWidth,fldPrecision)
    #%s represents a string, %d represents a signed interger, %f is a float
    fmt    = '%s: %s (%d.%d)'
    print fmt % values
print

# get feature count
nFeatures = arcpy.GetCount_management(fc2)[0]
print "number of features in", fc2, ":", nFeatures
print

# Using cursors to access features
#searchcursor iterates through the features in a feature class
#you then need to access the shape attribute to get the geom
cursor = arcpy.SearchCursor(fc2)
shapefieldname = desc.ShapeFieldName # get the name of the shape field 
#loop through all the features to get all the FID values. 
for row in cursor:
    print "feature id:", row.getValue("FID")
    #this gets the geom of the feature
    shape = row.getValue(shapefieldname)
    #get part retrives a list of values from the shape, using 0 is the list of points. 
    for pnt in shape.getPart(0):
         print pnt.X, pnt.Y
print

# buffer analysis
arcpy.MakeFeatureLayer_management(fc1, "sample_line_lyr") # the buffer analysis requires the input to be a feature layer
arcpy.Buffer_analysis("sample_line_lyr", "sample_line_buffer", "5000 feet") # can add more optional parameters
#if you dont specifiy a unit it will use the default unit

# select feature by location
arcpy.MakeFeatureLayer_management(fc2,"sample_polygon_lyr")
arcpy.MakeFeatureLayer_management("sample_line_buffer.shp","sample_line_buffer_lyr")
#will first need to make layers from the existing shapefiles
arcpy.SelectLayerByLocation_management("sample_polygon_lyr","INTERSECT","sample_line_buffer_lyr")
# For lab assignment, find the appropriate keyword from the official documentation.

# create output feature class 
sr = arcpy.Describe("sample_polygon_lyr").spatialReference
#same as template is the m value (??) and z value of the original file. sr means "same as reference"
arcpy.CreateFeatureclass_management(arcpy.env.workspace, "polygon_intersect.shp", "POLYGON", fc2, "SAME_AS_TEMPLATE", "SAME_AS_TEMPLATE", sr) # spatial reference
# add selection to output feature class
arcpy.Append_management("sample_polygon_lyr","polygon_intersect.shp")

#of note! If you run this on the parcel and powerline data it works, but you cant output those polygons to a new feature class, 
#the fix, probably need to make the working environment a geodatabase. 


featureType:    Simple
shapeFieldName: Shape
shapeType:      Polygon
extent:         -3.85350469301645 -1.15260914492379 -2.16642986308447 -0.442858578030368 NaN NaN NaN NaN
path:           .\data


The fields are:
FID: OID (4.0)
Shape: Geometry (0.0)
id: Integer (10.10)
name: String (10.0)

number of features in .\data\sample_polygon.shp : 2

feature id: 0
-3.85350469302 -0.87823272769
-3.77413961365 -0.44285857803
-3.22085163179 -0.569842705014
-3.25259766354 -1.00521685467
-3.85350469302 -0.87823272769
feature id: 1
-2.47708745946 -0.739910732225
-2.16642986308 -1.15260914492
-2.81949108757 -1.12539826057
-2.47708745946 -0.739910732225



<Result 'polygon_intersect.shp'>

In [9]:
##raster processing sample##
import arcpy
import os
arcpy.CheckOutExtension("Spatial")

arcpy.env.workspace = r"C:\Users\lfabr\OneDrive - UW-Madison\School\Madison\courses\2017-03-Fall\GEOG 378\labs\Lab7\data"

# describe raster
desc = arcpy.Describe('elevation.tif')
print desc.bandCount
print desc.compressionType
print desc.format
rows = desc.height
cols = desc.width

# calculate slope
inraster = arcpy.Raster("elevation.tif")
outraster = arcpy.sa.Slope("elevation.tif")

# hillshade
myShade = arcpy.sa.Hillshade("elevation.tif", 270, 45)  #sun az 270, elev 45

#get min and max
arcpy.CalculateStatistics_management(inraster)
MINResult = arcpy.GetRasterProperties_management(inraster, "MINIMUM")
zMin = float(MINResult.getOutput(0))
MAXResult = arcpy.GetRasterProperties_management(inraster, "MAXIMUM")
zMax = float(MAXResult.getOutput(0))

# create output
outName = 'SlopeColored.tif'
if os.path.exists(outName):
    #if this file allready exits remove the file first
     os.remove(outName)

#create raster of scaled elevations
f = ((inraster -zMin)/(zMax-zMin))
#create the red raster
R = 255* (inraster/inraster)
#green and blue raster values are identical
G = f * 255
B = f * 255
 
#Compose single band datasets to a TIFF format raster file
arcpy.CompositeBands_management(str(R)+';'+str(G)+';'+str(B),outName)


1
None
TIFF


ExecuteError: ERROR 999999: Error executing function.
Failed to delete raster dataset
Failed to execute (CompositeBands).
